In [1]:
import os
from notion_client import Client

notion = Client(auth=os.getenv("NOTION_TOKEN"))
recipe_book_id = "7381d2928f9745d484a9322122760a00"
igrd_id = '01dfa2430d6c4908b7fa6a78deb6e037'

In [2]:
def get_database(id):
    full_or_partial_pages = notion.databases.query(
        database_id=id
    )
    if full_or_partial_pages['has_more']:
        raise Warning('Problème avec la pagiation')
    else:
        database = full_or_partial_pages
    return database.get('results')

In [9]:
def get_url_dict(database_id):
    """
    Return Dict id:url
    """
    database = get_database(database_id)
    url_dict = {}
    for page in database:
        try:
            page_id = page['id']
            url = page['properties']['URL']['url']
            if url[:5] == "https":
                url_dict[page_id] = url
            else:
                print(f"Skipped wrong URL : {url}")
        except KeyError:
            print("Key error")
    return url_dict
f'Found {len(get_url_dict(recipe_book_id))} url'

Skipped wrong URL : oui


'Found 23 url'

In [4]:
def get_existing_irgd(database):
    """
    Return Dict : id:irgd
    """
    igrd_dict = {}
    for page in database:
        try:
            igrd_dict[page['id']] = page['properties']['Name']['title'][0]['text']['content']
        except KeyError:
            print('Key error mais très étrange')
    return igrd_dict
get_existing_irgd(get_database(igrd_id))

{'c5d6334e-5469-4397-b8d0-c6d5700016ad': 'HEYYY',
 '97c5df5e-519f-4db1-aeb6-95e794195a22': 'HEYYY',
 '21798723-d748-4bfb-9e33-e49c11f90eed': 'oui',
 'f04fbbde-9574-4845-92dd-c93696290d61': 'Leek',
 'fb87177a-e96e-44f9-b4b0-1344613a6398': 'Tofu'}

In [5]:
def add_igrd_to_recipe(recipe_id, igrd_id):
    """
    Add ingredient to the recipe
    """
    igrd_prop = notion.pages.retrieve(recipe_id)['properties']['Ingredients']
    igrd_prop['relation'].append({'id': igrd_id})
    notion.pages.update(page_id=recipe_id, properties={
            "Ingredients": igrd_prop
    })
# add_igrd_to_recipe('2c7d1976-0fea-4db9-b054-d9581f6503fc', 'fb87177a-e96e-44f9-b4b0-1344613a6398')


def add_igrd_to_database(database_id, name):
    """
    Add ingredient to the ingredient database
    """
    notion.pages.create(
        parent={
            "type": "database_id",
            "database_id": database_id
        },
        properties={
            'Name': {
                'id': 'title',
                'type': 'title',
                'title': [{'type': 'text',
                'text': {'content': name, 'link': None},
                'annotations': {'bold': False,
                'italic': False,
                'strikethrough': False,
                'underline': False,
                'code': False,
                'color': 'default'},
                'plain_text': 'Mocha Hazelnut Oatmeal',
                'href': None
                }]
            }
        }
    )
# add_igrd_to_database(igrd_id, 'HEYYY')


In [6]:
# notion.pages.retrieve('2c7d1976-0fea-4db9-b054-d9581f6503fc')['properties']

# Scrapping

In [17]:
import requests
from bs4 import BeautifulSoup

def get_recipe_igrd_from_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    igrd_div = soup.find('div', 'ingredientlist')
    if igrd_div == None:
        print("No igredients found")
        return None
    igrd_list = igrd_div.find_all("span", "ingredient-name-text")
    igrd_list = list(map(lambda x: x.string, igrd_list))
    return igrd_list

def clean_igrd_list():
    get_existing_irgd()

# BOOM Fusion

In [22]:
url_id_dict = get_url_dict(recipe_book_id)
for url in url_id_dict.values():
    if url.find('pickuplimes') == -1:
        continue
    else:
        print(url)
        irgd_list = get_recipe_igrd_from_url(url)
        print(irgd_list)
        

Skipped wrong URL : oui
https://www.pickuplimes.com/recipe/turkish-red-lentil-soup-26
['vegetable oil', 'medium onion', 'water', 'vegetable bouillon cube', 'dry red lentils', 'medium potato', 'paprika powder', 'onion powder', 'garlic powder', 'balsamic vinegar', 'ground black pepper', 'salt', 'unsweetened coconut yogurt', 'slices whole wheat bread']
https://www.pickuplimes.com/recipe/grilled-red-pepper-pesto-pasta-314
['olive oil', 'red bell peppers', 'orange bell pepper', 'salt', 'ground black pepper', 'cloves garlic', 'medium zucchini', 'dry spaghetti noodles', 'sun-dried tomatoes in oil', 'roasted walnuts', 'frozen green peas']
https://www.pickuplimes.com/recipe/coconut-tofu-thai-red-curry-255
['vegetable oil', 'medium shallots', 'cloves garlic', 'medium potatoes', 'Thai red curry paste', 'water', 'uncooked basmati rice', 'pre-seasoned tofu strips', 'red bell peppers', 'small broccoli', 'canned full-fat coconut milk', 'sodium-reduced\xa0soy sauce', 'sambal oelek', 'roasted cashews']

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No igredients found
None
https://www.pickuplimes.com/recipe/roasted-vegetable-salad-with-crispy-chickpeas-596
['cooked chickpeas', 'vegetable oil', 'burrito seasoning', 'medium potatoes', 'medium carrots', 'cloves garlic', 'vegetable oil', 'burrito seasoning', 'medium onions', 'fresh spinach', 'frozen green peas', 'unsweetened soy yogurt', 'salt', 'ground black pepper']
https://www.pickuplimes.com/recipe/macaroni-salad-with-lentil-quesadillas-592
['canned diced tomatoes', 'medium onion', 'clove garlic', 'salt', 'ground black pepper', 'dry whole wheat macaroni', 'frozen green peas', 'medium carrot', 'unsweetened soy yogurt', 'minced onion', 'burrito seasoning', 'vegetable bouillon cube', 'ground black pepper', 'dry green lentils', 'canned crushed tomatoes', 'burrito seasoning', 'large (25 cm) soft tortillas', 'vegetable oil']
https://www.pickuplimes.com/recipe/make-ahead-freezer-burritos-330
['soy veggie ground', 'Mexican spice mix', 'large (25 cm) soft tortillas', 'refried beans', 'red

In [21]:
"https://www.pickuplimes.com/recipe/turkish-red-lentil-soup-26".find('fdsf')

-1